<a href="https://colab.research.google.com/github/urvashipathak/ML/blob/main/6_Covid_Radiography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive

/content/drive/MyDrive


In [ ]:
!ls

 16178749727014456986985512586592.jpg
 16178771184991319007604593303214.jpg
 161787729111614586703938565840.jpg
 16178777667891400210872109987637.jpg
 16180383159211491476343439806047.jpg
'Aayush Panchal.NPTEL.pdf'
'Aayush Panchal.pdf'
'Alexa Skill set(Aayush Panchal 0827EC191001).docx'
'All students.xlsx'
 archive.zip
'Bajaj Finserv Health - Data Science - Qualifier 1 Challenge - MP Campus Pool (1).gdoc'
'Bajaj Finserv Health - Data Science - Qualifier 1 Challenge - MP Campus Pool (1).pdf'
'Bajaj Finserv Health - Data Science - Qualifier 1 Challenge - MP Campus Pool.gdoc'
'Bajaj Finserv Health - Data Science - Qualifier 1 Challenge - MP Campus Pool.pdf'
 bank_marketing.csv
'Certificate for Aayush Panchal for "Feedback for: '\''Emerging Tec...".pdf'
'Certificate for Aayush Panchal for "Feedback for '\''Image Enhance...".pdf'
'Certificate for Aayush Panchal for "National level Online E-Qui...".png'
 Classroom
'CN assignment 1 and 2 (0827EC191001).pdf'
'Colab Notebooks'
'Consult Form(082

In [ ]:
!unzip archive.zip

Archive:  archive.zip
replace COVID-19_Radiography_Dataset/COVID.metadata.xlsx? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pip install split_folders

In [ ]:
import splitfolders
splitfolders.ratio("/content/drive/MyDrive/COVID-19_Radiography_Dataset/COVID", output="split", seed=1337, ratio=(.8, .1, .1), group_prefix=None)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Normalize training and validation data in the range of 0 to 1
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Read the training sample and set the batch size 
train_generator = train_datagen.flow_from_directory(
        'split/train/',
        target_size=(128, 128),
        batch_size=8,
        seed=100,
        class_mode='categorical')

# Read Validation data from directory and define target size with batch size
validation_generator = validation_datagen.flow_from_directory(
        'split/val/',
        target_size=(128, 128),
        batch_size=8,
        class_mode='categorical',
        seed=1000,
        shuffle=False)

test_generator = test_datagen.flow_from_directory(
        'split/test/',
        target_size=(128, 128),
        batch_size=8,
        seed=500,
        class_mode='categorical',
        shuffle=False)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
inputs = keras.Input(shape=(128, 128,3))
x = layers.Flatten()(inputs)
x = layers.Dense(32, activation="relu")(x)
x = layers.Dense(64, activation='relu')(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 flatten_1 (Flatten)         (None, 49152)             0         
                                                                 
 dense_3 (Dense)             (None, 32)                1572896   
                                                                 
 dense_4 (Dense)             (None, 64)                2112      
                                                                 
 dense_5 (Dense)             (None, 2)                 130       
                                                                 
Total params: 1,575,138
Trainable params: 1,575,138
Non-trainable params: 0
_________________________________________________________________


In [ ]:
! pip install tensorflow

     |████████████████████████████████| 462 kB 5.1 MB/s 


In [ ]:
from tensorflow.keras.optimizers import Adam

adam = Adam(learning_rate=0.0001)
# We are going to use accuracy metrics and cross entropy loss as performance parameters
model.compile(adam, loss='categorical_crossentropy', metrics=['acc'])
# Train the model 
history = model.fit(train_generator, 
      steps_per_epoch=train_generator.samples/train_generator.batch_size,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=validation_generator.samples/validation_generator.batch_size,verbose=1)

Epoch 1/100
723/723 [==============================] - 33s 45ms/step - loss: 0.0153 - acc: 0.9946 - val_loss: 0.0033 - val_acc: 0.9986
Epoch 2/100
723/723 [==============================] - 33s 46ms/step - loss: 0.0039 - acc: 0.9991 - val_loss: 0.0025 - val_acc: 0.9986
Epoch 3/100
723/723 [==============================] - 31s 43ms/step - loss: 0.0022 - acc: 0.9995 - val_loss: 5.0159e-05 - val_acc: 1.0000
Epoch 4/100
723/723 [==============================] - 33s 46ms/step - loss: 4.3380e-04 - acc: 0.9998 - val_loss: 1.5907e-04 - val_acc: 1.0000
Epoch 5/100
723/723 [==============================] - 36s 50ms/step - loss: 3.9957e-05 - acc: 1.0000 - val_loss: 1.3944e-04 - val_acc: 1.0000
Epoch 6/100
723/723 [==============================] - 32s 45ms/step - loss: 2.2304e-05 - acc: 1.0000 - val_loss: 1.1393e-04 - val_acc: 1.0000
Epoch 7/100
723/723 [==============================] - 34s 46ms/step - loss: 1.5847e-05 - acc: 1.0000 - val_loss: 1.0013e-04 - val_acc: 1.0000
Epoch 8/100
723/723

In [ ]:
model.save('covid_classification.h5')

In [ ]:
from tensorflow.keras import models
model = models.load_model('covid_classification.h5')

In [ ]:
model.save_weights('covid_classification_weights.h5')

In [ ]:
model.load_weights('covid_classification_weights.h5')

In [ ]:
train_acc = history.history['acc']
val_acc = history.history['val_acc']
train_loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
import matplotlib.pyplot as plt
epochs = range(len(train_acc)) 
plt.plot(epochs, train_acc, 'b', label='Training Accuracy')
plt.plot(epochs, val_acc, 'r', label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.figure()
plt.show()

plt.plot(epochs, train_loss, 'b', label='Training Loss')
plt.plot(epochs, val_loss, 'r', label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
test_output= model.evaluate(test_generator, steps=test_generator.samples/test_generator.batch_size, verbose=1)
print(test_output)
print(model.metrics_names)